In [3]:
import random
import numpy as np
from utils import get_logs, parse_logs, clean_data, LogParser
from model import NgramModel


In [4]:
logs = parse_logs(get_logs()) 
log_parser = LogParser(logs)
all_keys, white_keys, black_keys, draw_keys = log_parser.parse_logs()


In [5]:
def extract_words(logs, key):
    pieces = []
    rows = logs[key]
    rows_count = len(rows)
    for i, row in enumerate(rows):
        if len(row) == 3:
            pieces.extend(row[1:3])
    return pieces

In [6]:
words = [extract_words(logs, key) for key in all_keys]
word_test = [extract_words(logs, key) for key in all_keys][-1]

In [10]:
words

[['wG1',
  '.',
  'bG1',
  '\\wG1',
  'wA1',
  'wG1\\',
  'bG2',
  '\\bG1',
  'wQ',
  'wG1-',
  'bQ',
  '-bG1',
  'wA1',
  'bG2/',
  'bA1',
  '-bG2',
  'wG2',
  'wG1\\',
  'bA1',
  '/wG2',
  'wS1',
  '\\wA1',
  'bA2',
  '-bA1',
  'wS1',
  '-bQ',
  'bS1',
  'bA1\\',
  'wA1',
  '-bA2',
  'bS1',
  'wQ-',
  'wA2',
  '\\wS1',
  'bB1',
  'bG2-',
  'wA2',
  'bB1/',
  'bA3',
  'bA2\\',
  'wS2',
  '/wA1',
  'bA3',
  '-wS2',
  'wA3',
  '\\wS1',
  'bS2',
  'bS1/',
  'wA3',
  '\\bA3',
  'bG3',
  'bA2\\',
  'wG3',
  'wA3/',
  'bG3',
  'wQ/',
  'wG3',
  '-bA3',
  'bS2',
  'wG2-',
  'wB1',
  '\\wS1',
  'bB2',
  'bG3-',
  'wA3',
  'bB2-',
  'bG2',
  'bB1-',
  'wG1',
  'bA1-',
  'bS1',
  'wA3-',
  'wB1',
  'wS1',
  'bS2',
  'wA3\\',
  'wB1',
  'bQ',
  'bS1',
  'bG2-',
  'wA2',
  'bS2\\',
  'bS1',
  '\\bB1',
  'wG1',
  'bA3/',
  'bS1',
  '\\wS1',
  'wG1',
  'wG3\\',
  'bS1',
  '\\bB1',
  'wG3',
  'wS2-',
  'bS1',
  '\\wS1',
  'wA1',
  '\\bS1',
  'wB2',
  'wS1\\',
  'wB2',
  'wB1\\',
  'wA1',
  'bS1-',
 

In [11]:
def extract_ngrams(tokens, n):
        if n == 1:
            tokens_extended = ['s'] + tokens    
        else:
            tokens_extended = ['s'] * (n - 1) + tokens 
        ngrams = []
        for i in range(len(tokens_extended) - n):
            if n == 1:
                ngram = tokens_extended[i:i + n][0]
            else:
                ngram = tuple(tokens_extended[i:i + n])
            ngrams.append(ngram)
        ngrams = np.array(ngrams)
        return ngrams

In [12]:
def concatenate_ngrams(logs, all_keys, n):
    ngrams = [extract_ngrams(extract_words(logs, key), n) for key in all_keys]
    concatenated_array = np.concatenate(ngrams)
    return concatenated_array

In [13]:
one = concatenate_ngrams(logs, all_keys, 1)
two = concatenate_ngrams(logs, all_keys, 2)
three = concatenate_ngrams(logs, all_keys, 3)
four = concatenate_ngrams(logs, all_keys, 4)


In [14]:
two

array([['s', 'wG1'],
       ['wG1', '.'],
       ['.', 'bG1'],
       ...,
       ['bQ-', 'bG3'],
       ['bG3', 'wA2-'],
       ['wA2-', 'wB1']], dtype='<U4')

In [23]:
import torch
from torch import  nn

In [43]:
unique_rows = np.unique(two, axis=0)
unique_rows = [row[0] + row[1] for row in  unique_rows]

ix_w = {w:i for i, w in enumerate(unique_rows)}

In [44]:
ix_w

{'-bA1bA1': 0,
 '-bA1bA2': 1,
 '-bA1bA3': 2,
 '-bA1bB1': 3,
 '-bA1bB2': 4,
 '-bA1bG1': 5,
 '-bA1bG2': 6,
 '-bA1bG3': 7,
 '-bA1bS1': 8,
 '-bA1bS2': 9,
 '-bA1wA1': 10,
 '-bA1wA2': 11,
 '-bA1wA3': 12,
 '-bA1wB1': 13,
 '-bA1wB2': 14,
 '-bA1wG1': 15,
 '-bA1wG2': 16,
 '-bA1wG3': 17,
 '-bA1wQ': 18,
 '-bA1wS1': 19,
 '-bA1wS2': 20,
 '-bA2bA1': 21,
 '-bA2bA2': 22,
 '-bA2bA3': 23,
 '-bA2bB1': 24,
 '-bA2bB2': 25,
 '-bA2bG1': 26,
 '-bA2bG2': 27,
 '-bA2bG3': 28,
 '-bA2bS1': 29,
 '-bA2bS2': 30,
 '-bA2wA1': 31,
 '-bA2wA2': 32,
 '-bA2wA3': 33,
 '-bA2wB1': 34,
 '-bA2wB2': 35,
 '-bA2wG2': 36,
 '-bA2wG3': 37,
 '-bA2wS1': 38,
 '-bA2wS2': 39,
 '-bA3bA1': 40,
 '-bA3bA2': 41,
 '-bA3bA3': 42,
 '-bA3bB1': 43,
 '-bA3bB2': 44,
 '-bA3bG1': 45,
 '-bA3bG2': 46,
 '-bA3bG3': 47,
 '-bA3bS1': 48,
 '-bA3bS2': 49,
 '-bA3wA1': 50,
 '-bA3wA2': 51,
 '-bA3wA3': 52,
 '-bA3wB1': 53,
 '-bA3wB2': 54,
 '-bA3wG2': 55,
 '-bA3wG3': 56,
 '-bA3wQ': 57,
 '-bA3wS2': 58,
 '-bB1bA1': 59,
 '-bB1bA2': 60,
 '-bB1bA3': 61,
 '-bB1bB1': 62,
 '-b

In [51]:
embeds = nn.Embedding(len(ix_w), 10)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([ix_w["\\wB1bB2"]], dtype=torch.long)
test_embed = embeds(lookup_tensor)
print(test_embed)

tensor([[-0.8967, -0.0133,  0.5902,  0.0769, -0.7321, -1.7270, -0.8214,  0.4284,
          0.8253, -0.7736]], grad_fn=<EmbeddingBackward0>)


In [52]:
import matplotlib.pyplot as plt

In [54]:
import numpy as np 

In [19]:

model = NgramModel(4)
model.update(four)

In [20]:

model.calculate_probs()


{('s', 's', 's', 'wG1'): 0.8134328358208955,
 ('s', 's', 'wG1', '.'): 1.0,
 ('s', 'wG1', '.', 'bG1'): 0.7981651376146789,
 ('wG1', '.', 'bG1', '\\wG1'): 0.022988505747126436,
 ('.', 'bG1', '\\wG1', 'wA1'): 1.0,
 ('bG1', '\\wG1', 'wA1', 'wG1\\'): 1.0,
 ('\\wG1', 'wA1', 'wG1\\', 'bG2'): 0.6,
 ('wA1', 'wG1\\', 'bG2', '\\bG1'): 0.6,
 ('wG1\\', 'bG2', '\\bG1', 'wQ'): 1.0,
 ('bG2', '\\bG1', 'wQ', 'wG1-'): 1.0,
 ('\\bG1', 'wQ', 'wG1-', 'bQ'): 1.0,
 ('wQ', 'wG1-', 'bQ', '-bG1'): 0.36363636363636365,
 ('wG1-', 'bQ', '-bG1', 'wA1'): 0.8,
 ('bQ', '-bG1', 'wA1', 'bG2/'): 0.14285714285714285,
 ('-bG1', 'wA1', 'bG2/', 'bA1'): 1.0,
 ('wA1', 'bG2/', 'bA1', '-bG2'): 0.5,
 ('bG2/', 'bA1', '-bG2', 'wG2'): 1.0,
 ('bA1', '-bG2', 'wG2', 'wG1\\'): 1.0,
 ('-bG2', 'wG2', 'wG1\\', 'bA1'): 1.0,
 ('wG2', 'wG1\\', 'bA1', '/wG2'): 1.0,
 ('wG1\\', 'bA1', '/wG2', 'wS1'): 1.0,
 ('bA1', '/wG2', 'wS1', '\\wA1'): 1.0,
 ('/wG2', 'wS1', '\\wA1', 'bA2'): 0.5,
 ('wS1', '\\wA1', 'bA2', '-bA1'): 1.0,
 ('\\wA1', 'bA2', '-bA1', 

In [13]:
test_moves = word_test
model.perplexity(test_moves)

perplexity: 0
perplexity: 0
perplexity: 0
perplexity: 0
perplexity: 0
perplexity: 0
ngram probability: 1.0, perplexity: 1.0
perplexity: 1.0
ngram probability: 0.3333333333333333, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram probability: 1.0, perplexity: 1.169930812758687
perplexity: 1.2
ngram

ZeroDivisionError: 0.0 cannot be raised to a negative power